---
title: "Foundation Model Architectures"
subtitle: "LLMs vs. Geospatial Foundation Models (GFMs)"
jupyter: geoai
format: html
---


## Introduction to Foundation Model Architectures

Foundation models are large-scale models trained on diverse data that can be adapted to various downstream tasks. This cheatsheet compares Language Model (LLM) and Geospatial Foundation Model (GFM) development pipelines.


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModel, AutoConfig

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Evolution from AI to Transformers

### Key Historical Milestones

In [ ]:
# Timeline of key developments
timeline = {
    "1950s-1990s": "Symbolic AI, early neural networks",
    "2012": "Deep learning breakthrough (ImageNet)",
    "2017": "Transformers ('Attention Is All You Need')",
    "2018-2020": "BERT/GPT families emerge",
    "2021-2024": "Scaling laws, instruction tuning, multimodality"
}

print("AI/ML → Transformers Timeline:")
for year, development in timeline.items():
    print(f"{year}: {development}")

### Transformer Architecture Essentials

In [ ]:
class SimpleTransformerBlock(nn.Module):
    """Simplified transformer block to illustrate key components"""
    
    def __init__(self, embed_dim=768, num_heads=8, mlp_ratio=4):
        super().__init__()
        self.embed_dim = embed_dim
        
        # Multi-head attention
        self.attention = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)
        
        # Feed-forward network
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, int(embed_dim * mlp_ratio)),
            nn.GELU(),
            nn.Linear(int(embed_dim * mlp_ratio), embed_dim)
        )
        
        # Layer normalization
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
    
    def forward(self, x):
        # Self-attention with residual connection
        attn_out, _ = self.attention(x, x, x)
        x = self.norm1(x + attn_out)
        
        # MLP with residual connection
        mlp_out = self.mlp(x)
        x = self.norm2(x + mlp_out)
        
        return x

# Example transformer block
transformer_block = SimpleTransformerBlock(embed_dim=768, num_heads=8)
sample_input = torch.randn(2, 100, 768)  # [batch_size, seq_len, embed_dim]
output = transformer_block(sample_input)

print(f"Input shape: {sample_input.shape}")
print(f"Output shape: {output.shape}")
print(f"Parameters: {sum(p.numel() for p in transformer_block.parameters()):,}")

## 9-Step Development Pipeline Comparison

### LLM Development Pipeline

In [ ]:
llm_pipeline = {
    "1. Data Preparation": "Text corpora, deduplication, quality filtering, mixing ratios",
    "2. Tokenization": "BPE, vocabulary construction, special tokens",
    "3. Architecture": "GPT/BERT variants, depth/width scaling, context length",
    "4. Pretraining Objective": "Next-token prediction, masked language modeling",
    "5. Training Loop": "Optimizers, LR schedules, mixed precision, gradient clipping",
    "6. Evaluation": "Perplexity, downstream task probing, benchmarks",
    "7. Pretrained Weights": "Model hubs, tokenizer alignment, loading utilities",
    "8. Finetuning": "Task-specific heads, PEFT methods, instruction tuning",
    "9. Deployment": "API serving, KV caching, inference optimization"
}

print("LLM Development Pipeline:")
for step, description in llm_pipeline.items():
    print(f"{step}: {description}")

### GFM Development Pipeline

In [ ]:
gfm_pipeline = {
    "1. Data Preparation": "Multi-spectral data, georegistration, tiling, cloud masking",
    "2. Tokenization": "Patch-based, continuous embeddings, 2D/temporal positions",
    "3. Architecture": "ViT encoders, spatial/temporal attention, memory constraints",
    "4. Pretraining Objective": "Masked patch reconstruction, contrastive learning",
    "5. Training Loop": "Cloud masks, mixed precision, gradient accumulation",
    "6. Evaluation": "Reconstruction metrics, linear probing, generalization",
    "7. Pretrained Weights": "Prithvi, SatMAE, adapter loading, band alignment",
    "8. Finetuning": "Task heads, PEFT, few-shot learning for limited labels",
    "9. Deployment": "Tiling inference, geospatial APIs, batch processing"
}

print("\nGFM Development Pipeline:")
for step, description in gfm_pipeline.items():
    print(f"{step}: {description}")

## Step-by-Step Detailed Comparison

### 1. Data Preparation Differences

In [ ]:
def demonstrate_data_preparation():
    """Show key differences in data preparation"""
    
    # LLM data preparation simulation
    print("LLM Data Preparation:")
    print("- Text scraping from web, books, code")
    print("- Deduplication algorithms")
    print("- Quality filtering (language detection, toxicity)")
    print("- Data mixing ratios optimization")
    
    # Simulate text preprocessing
    sample_texts = [
        "The quick brown fox jumps over the lazy dog.",
        "Machine learning is transforming many industries.",
        "Climate change requires urgent global action."
    ]
    
    # Basic tokenization simulation
    vocab = set()
    for text in sample_texts:
        vocab.update(text.lower().replace('.', '').split())
    
    print(f"\nSample vocabulary size: {len(vocab)}")
    print(f"Sample tokens: {list(vocab)[:10]}")
    
    print("\n" + "="*50)
    
    # GFM data preparation simulation
    print("GFM Data Preparation:")
    print("- Sensor calibration and atmospheric correction")
    print("- Georegistration and projection alignment")
    print("- Cloud masking and quality assessment")
    print("- Temporal compositing and gap filling")
    
    # Simulate satellite data preprocessing
    np.random.seed(42)
    
    # Simulate multi-spectral satellite patch
    patch_size = 64
    num_bands = 6
    satellite_patch = np.random.randint(0, 4096, (num_bands, patch_size, patch_size))
    
    # Simulate cloud mask
    cloud_mask = np.random.random((patch_size, patch_size)) > 0.8
    
    # Apply atmospheric correction (simplified)
    corrected_patch = satellite_patch.astype(np.float32) / 4095.0
    corrected_patch[cloud_mask] = np.nan  # Mask cloudy pixels
    
    print(f"\nSatellite patch shape: {satellite_patch.shape}")
    print(f"Cloud coverage: {cloud_mask.sum() / cloud_mask.size * 100:.1f}%")
    print(f"Valid pixels: {(~np.isnan(corrected_patch[0])).sum():,}")

demonstrate_data_preparation()

### 2. Tokenization Approaches

In [ ]:
def compare_tokenization():
    """Compare LLM vs GFM tokenization approaches"""
    
    print("LLM Tokenization (Discrete):")
    # Simulate BPE tokenization
    vocab_size, embed_dim = 50000, 768
    
    # Sample token sequence
    token_ids = torch.tensor([1, 15, 234, 5678, 2])  # [CLS, word1, word2, word3, SEP]
    
    # Embedding lookup
    embedding_layer = nn.Embedding(vocab_size, embed_dim)
    token_embeddings = embedding_layer(token_ids)
    
    print(f"Token IDs: {token_ids.tolist()}")
    print(f"Token embeddings shape: {token_embeddings.shape}")
    print(f"Vocabulary size: {vocab_size:,}")
    
    print("\n" + "-"*30)
    
    print("GFM Tokenization (Continuous Patches):")
    # Simulate patch-based tokenization
    patch_size = 16
    num_bands = 6
    embed_dim = 768
    
    # Create sample patches
    num_patches = 4
    patch_dim = patch_size * patch_size * num_bands
    patches = torch.randn(num_patches, patch_dim)
    
    # Linear projection (continuous "tokenization")
    patch_projection = nn.Linear(patch_dim, embed_dim)
    patch_embeddings = patch_projection(patches)
    
    print(f"Patch dimensions: {patch_size}x{patch_size}x{num_bands} = {patch_dim}")
    print(f"Patch embeddings shape: {patch_embeddings.shape}")
    print(f"No discrete vocabulary - continuous projection")
    
    return token_embeddings, patch_embeddings

token_emb, patch_emb = compare_tokenization()

### 3. Architecture Comparison

In [ ]:
class LLMArchitecture(nn.Module):
    """Simplified LLM architecture (GPT-style)"""
    
    def __init__(self, vocab_size=50000, embed_dim=768, num_layers=12, num_heads=12):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.positional_encoding = nn.Embedding(2048, embed_dim)  # Max sequence length
        
        self.layers = nn.ModuleList([
            SimpleTransformerBlock(embed_dim, num_heads) 
            for _ in range(num_layers)
        ])
        
        self.ln_final = nn.LayerNorm(embed_dim)
        self.output_head = nn.Linear(embed_dim, vocab_size)
    
    def forward(self, input_ids):
        seq_len = input_ids.shape[1]
        positions = torch.arange(seq_len, device=input_ids.device)
        
        # Token + positional embeddings
        x = self.embedding(input_ids) + self.positional_encoding(positions)
        
        # Transformer layers
        for layer in self.layers:
            x = layer(x)
        
        x = self.ln_final(x)
        logits = self.output_head(x)
        
        return logits

class GFMArchitecture(nn.Module):
    """Simplified GFM architecture (ViT-style)"""
    
    def __init__(self, patch_size=16, num_bands=6, embed_dim=768, num_layers=12, num_heads=12):
        super().__init__()
        self.patch_size = patch_size
        self.num_bands = num_bands
        
        # Patch embedding
        patch_dim = patch_size * patch_size * num_bands
        self.patch_embedding = nn.Linear(patch_dim, embed_dim)
        
        # 2D positional embedding
        self.pos_embed_h = nn.Embedding(100, embed_dim // 2)  # Height positions
        self.pos_embed_w = nn.Embedding(100, embed_dim // 2)  # Width positions
        
        self.layers = nn.ModuleList([
            SimpleTransformerBlock(embed_dim, num_heads) 
            for _ in range(num_layers)
        ])
        
        self.ln_final = nn.LayerNorm(embed_dim)
    
    def forward(self, patches, patch_positions):
        batch_size, num_patches, patch_dim = patches.shape
        
        # Patch embeddings
        x = self.patch_embedding(patches)
        
        # 2D positional embeddings
        pos_h, pos_w = patch_positions[:, :, 0], patch_positions[:, :, 1]
        pos_emb = torch.cat([
            self.pos_embed_h(pos_h),
            self.pos_embed_w(pos_w)
        ], dim=-1)
        
        x = x + pos_emb
        
        # Transformer layers
        for layer in self.layers:
            x = layer(x)
        
        x = self.ln_final(x)
        return x

# Compare architectures
llm_model = LLMArchitecture(vocab_size=10000, embed_dim=384, num_layers=6, num_heads=6)
gfm_model = GFMArchitecture(patch_size=16, num_bands=6, embed_dim=384, num_layers=6, num_heads=6)

llm_params = sum(p.numel() for p in llm_model.parameters())
gfm_params = sum(p.numel() for p in gfm_model.parameters())

print("Architecture Comparison:")
print(f"LLM parameters: {llm_params:,}")
print(f"GFM parameters: {gfm_params:,}")

# Test forward passes
sample_tokens = torch.randint(0, 10000, (2, 50))  # [batch_size, seq_len]
sample_patches = torch.randn(2, 16, 16*16*6)  # [batch_size, num_patches, patch_dim]
sample_positions = torch.randint(0, 10, (2, 16, 2))  # [batch_size, num_patches, 2]

llm_output = llm_model(sample_tokens)
gfm_output = gfm_model(sample_patches, sample_positions)

print(f"\nLLM output shape: {llm_output.shape}")
print(f"GFM output shape: {gfm_output.shape}")

### 4. Pretraining Objectives

In [ ]:
def demonstrate_pretraining_objectives():
    """Compare pretraining objectives for LLMs vs GFMs"""
    
    print("LLM Pretraining Objectives:")
    print("1. Next-Token Prediction (GPT-style)")
    print("2. Masked Language Modeling (BERT-style)")
    
    # Simulate next-token prediction
    sequence = torch.tensor([[1, 2, 3, 4, 5]])
    targets = torch.tensor([[2, 3, 4, 5, 6]])  # Shifted by one
    
    # Mock logits
    vocab_size = 1000
    logits = torch.randn(1, 5, vocab_size)
    
    # Cross-entropy loss
    ce_loss = nn.CrossEntropyLoss()
    next_token_loss = ce_loss(logits.view(-1, vocab_size), targets.view(-1))
    
    print(f"Next-token prediction loss: {next_token_loss.item():.4f}")
    
    print("\n" + "-"*30)
    
    print("GFM Pretraining Objectives:")
    print("1. Masked Patch Reconstruction (MAE-style)")
    print("2. Contrastive Learning (optional)")
    
    # Simulate masked patch reconstruction
    batch_size, num_patches, patch_dim = 2, 64, 768
    original_patches = torch.randn(batch_size, num_patches, patch_dim)
    
    # Random masking
    mask_ratio = 0.75
    num_masked = int(num_patches * mask_ratio)
    
    # Create random mask
    mask = torch.zeros(batch_size, num_patches, dtype=torch.bool)
    for i in range(batch_size):
        masked_indices = torch.randperm(num_patches)[:num_masked]
        mask[i, masked_indices] = True
    
    # Reconstruction loss (simplified)
    reconstructed_patches = torch.randn_like(original_patches)
    reconstruction_loss = nn.MSELoss()(
        reconstructed_patches[mask], 
        original_patches[mask]
    )
    
    print(f"Mask ratio: {mask_ratio:.1%}")
    print(f"Masked patches per sample: {num_masked}")
    print(f"Reconstruction loss: {reconstruction_loss.item():.4f}")

demonstrate_pretraining_objectives()

## Scaling and Evolution

### Parameter Scaling Comparison

In [ ]:
def compare_scaling_trends():
    """Compare scaling trends between LLMs and GFMs"""
    
    # LLM scaling milestones
    llm_milestones = {
        'GPT-1': 117e6,
        'BERT-Base': 110e6,
        'GPT-2': 1.5e9,
        'GPT-3': 175e9,
        'PaLM': 540e9,
        'GPT-4': 1000e9  # Estimated
    }
    
    # GFM scaling examples
    gfm_milestones = {
        'SatMAE-Base': 86e6,
        'Prithvi-100M': 100e6,
        'Clay-v0.1': 139e6,
        'SatLas-Base': 300e6,
        'Scale-MAE': 600e6
    }
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # LLM scaling
    models = list(llm_milestones.keys())
    params = [llm_milestones[m]/1e9 for m in models]
    
    ax1.bar(models, params, color='skyblue', alpha=0.7)
    ax1.set_yscale('log')
    ax1.set_ylabel('Parameters (Billions)')
    ax1.set_title('LLM Parameter Scaling')
    ax1.tick_params(axis='x', rotation=45)
    
    # GFM scaling
    models = list(gfm_milestones.keys())
    params = [gfm_milestones[m]/1e6 for m in models]
    
    ax2.bar(models, params, color='lightcoral', alpha=0.7)
    ax2.set_ylabel('Parameters (Millions)')
    ax2.set_title('GFM Parameter Scaling')
    ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Context/Input scaling
    print("Context/Input Scaling:")
    print("\nLLMs:")
    print("- Context length: 512 → 2K → 8K → 128K+ tokens")
    print("- Training data: Web text, books, code (curated)")
    print("- Focus: Language understanding and generation")
    
    print("\nGFMs:")
    print("- Input bands: 3 (RGB) → 6+ (multispectral) → hyperspectral")
    print("- Spatial resolution: Various (10m to 0.3m)")
    print("- Temporal dimension: Single → time series → multi-temporal")
    print("- Focus: Earth observation and environmental monitoring")

compare_scaling_trends()

### Data Requirements and Constraints

In [ ]:
def compare_data_requirements():
    """Compare data requirements and constraints"""
    
    comparison = {
        "Data Volume": {
            "LLMs": "Terabytes of text (web crawls, books, code)",
            "GFMs": "Petabytes of satellite imagery (constrained by storage/IO)"
        },
        "Data Quality": {
            "LLMs": "Deduplication, toxicity filtering, language detection",
            "GFMs": "Cloud masking, atmospheric correction, sensor calibration"
        },
        "Preprocessing": {
            "LLMs": "Tokenization, sequence packing, attention masks",
            "GFMs": "Patch extraction, normalization, spatial alignment"
        },
        "Storage Format": {
            "LLMs": "Compressed text files, tokenized sequences",
            "GFMs": "Cloud-optimized formats (COG, Zarr), tiled storage"
        },
        "Access Patterns": {
            "LLMs": "Sequential text processing, random sampling",
            "GFMs": "Spatial/temporal queries, patch sampling, tiling"
        }
    }
    
    print("Data Requirements Comparison:")
    print("="*50)
    
    for aspect, details in comparison.items():
        print(f"\n{aspect}:")
        print(f"  LLMs: {details['LLMs']}")
        print(f"  GFMs: {details['GFMs']}")

compare_data_requirements()

## Implementation Examples

### Embedding Creation

In [ ]:
def demonstrate_embeddings():
    """Show embedding creation for both domains"""
    
    print("Text Embeddings (LLM):")
    # Simulate text tokenization and embedding
    text = "The forest shows signs of deforestation."
    tokens = text.lower().replace('.', '').split()
    
    # Create simple vocabulary
    vocab = {word: i for i, word in enumerate(set(tokens))}
    vocab['<PAD>'] = len(vocab)
    vocab['<UNK>'] = len(vocab)
    
    # Convert tokens to IDs
    token_ids = [vocab.get(token, vocab['<UNK>']) for token in tokens]
    token_tensor = torch.tensor(token_ids).unsqueeze(0)
    
    # Embedding layer
    embed_layer = nn.Embedding(len(vocab), 256)
    text_embeddings = embed_layer(token_tensor)
    
    print(f"Original text: {text}")
    print(f"Tokens: {tokens}")
    print(f"Token IDs: {token_ids}")
    print(f"Embeddings shape: {text_embeddings.shape}")
    
    print("\n" + "-"*40)
    
    print("Patch Embeddings (GFM):")
    # Simulate satellite patch processing
    patch_size = 16
    num_bands = 6
    
    # Create sample satellite patch
    satellite_patch = torch.randn(1, num_bands, patch_size, patch_size)
    
    # Reshape to patch format
    patch_flat = satellite_patch.view(1, num_bands * patch_size * patch_size)
    
    # Linear projection to embedding space
    patch_projection = nn.Linear(num_bands * patch_size * patch_size, 256)
    patch_embeddings = patch_projection(patch_flat)
    
    print(f"Original patch shape: {satellite_patch.shape}")
    print(f"Flattened patch shape: {patch_flat.shape}")
    print(f"Patch embeddings shape: {patch_embeddings.shape}")
    
    return text_embeddings, patch_embeddings

text_emb, patch_emb = demonstrate_embeddings()

### Positional Encoding Comparison

In [ ]:
def compare_positional_encodings():
    """Compare positional encoding strategies"""
    
    print("1D Positional Encoding (LLM):")
    
    def sinusoidal_positional_encoding(seq_len, embed_dim):
        """Create sinusoidal positional encodings"""
        pe = torch.zeros(seq_len, embed_dim)
        position = torch.arange(0, seq_len).unsqueeze(1).float()
        
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * 
                           -(np.log(10000.0) / embed_dim))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        return pe
    
    seq_len, embed_dim = 100, 256
    pos_encoding_1d = sinusoidal_positional_encoding(seq_len, embed_dim)
    
    print(f"1D positional encoding shape: {pos_encoding_1d.shape}")
    
    # Visualize positional encoding
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(pos_encoding_1d[:50, :50].T, cmap='RdBu', aspect='auto')
    plt.title('1D Positional Encoding (LLM)')
    plt.xlabel('Sequence Position')
    plt.ylabel('Embedding Dimension')
    plt.colorbar()
    
    print("\n2D Positional Encoding (GFM):")
    
    def create_2d_positional_encoding(height, width, embed_dim):
        """Create 2D positional encodings for spatial data"""
        pe = torch.zeros(height, width, embed_dim)
        
        # Create position grids
        y_pos = torch.arange(height).unsqueeze(1).repeat(1, width).float()
        x_pos = torch.arange(width).unsqueeze(0).repeat(height, 1).float()
        
        # Encode positions
        div_term = torch.exp(torch.arange(0, embed_dim//2, 2).float() * 
                           -(np.log(10000.0) / (embed_dim//2)))
        
        # Y (height) encoding
        pe[:, :, 0:embed_dim//4:2] = torch.sin(y_pos.unsqueeze(-1) * div_term)
        pe[:, :, 1:embed_dim//4:2] = torch.cos(y_pos.unsqueeze(-1) * div_term)
        
        # X (width) encoding
        pe[:, :, embed_dim//2:3*embed_dim//4:2] = torch.sin(x_pos.unsqueeze(-1) * div_term)
        pe[:, :, embed_dim//2+1:3*embed_dim//4:2] = torch.cos(x_pos.unsqueeze(-1) * div_term)
        
        return pe
    
    height, width = 8, 8
    pos_encoding_2d = create_2d_positional_encoding(height, width, embed_dim)
    
    print(f"2D positional encoding shape: {pos_encoding_2d.shape}")
    
    # Visualize 2D positional encoding
    plt.subplot(1, 2, 2)
    # Show first 64 dimensions as an 8x8 grid
    pos_2d_viz = pos_encoding_2d[:, :, :64].mean(dim=-1)
    plt.imshow(pos_2d_viz, cmap='RdBu', aspect='equal')
    plt.title('2D Positional Encoding (GFM)')
    plt.xlabel('Width')
    plt.ylabel('Height')
    plt.colorbar()
    
    plt.tight_layout()
    plt.show()

compare_positional_encodings()

## Course Mapping and Applications

### Weekly Course Structure

In [ ]:
course_mapping = {
    "Weeks 1-3": {
        "Focus": "Data → Attention → Architecture",
        "LLM Topics": ["Text preprocessing", "Tokenization", "Transformer blocks"],
        "GFM Topics": ["Satellite data", "Patch embedding", "Spatial attention"]
    },
    "Weeks 4-7": {
        "Focus": "Pretraining → Training → Evaluation → Integration",
        "LLM Topics": ["Language modeling", "Training loops", "Perplexity"],
        "GFM Topics": ["Masked reconstruction", "Cloud handling", "Linear probing"]
    },
    "Weeks 8-10": {
        "Focus": "Finetuning → Deployment → Synthesis",
        "LLM Topics": ["Instruction tuning", "PEFT", "API deployment"],
        "GFM Topics": ["Task heads", "Few-shot learning", "Geospatial inference"]
    }
}

print("Course Structure Mapping:")
print("="*50)

for period, details in course_mapping.items():
    print(f"\n{period} - {details['Focus']}")
    print(f"  LLM Focus: {', '.join(details['LLM Topics'])}")
    print(f"  GFM Focus: {', '.join(details['GFM Topics'])}")

### Key Takeaways

In [ ]:
key_differences = {
    "Data Nature": {
        "LLMs": "Discrete text tokens with semantic consistency",
        "GFMs": "Continuous pixel values requiring contextual interpretation"
    },
    "Tokenization": {
        "LLMs": "Vocabulary-based discrete mapping",
        "GFMs": "Patch-based continuous projection"
    },
    "Positional Info": {
        "LLMs": "1D sequence positions",
        "GFMs": "2D spatial + temporal positions"
    },
    "Training Objective": {
        "LLMs": "Next token prediction or masked language modeling",
        "GFMs": "Masked patch reconstruction or contrastive learning"
    },
    "Evaluation": {
        "LLMs": "Perplexity and downstream language tasks",
        "GFMs": "Reconstruction quality and spatial generalization"
    },
    "Deployment": {
        "LLMs": "Text generation with streaming and caching",
        "GFMs": "Spatial inference with tiling and batch processing"
    }
}

print("Key Architectural Differences:")
print("="*60)

for aspect, comparison in key_differences.items():
    print(f"\n{aspect}:")
    print(f"  LLMs: {comparison['LLMs']}")
    print(f"  GFMs: {comparison['GFMs']}")

## Further Reading and References

### Essential Papers

In [ ]:
references = {
    "Foundation Papers": [
        "Attention Is All You Need (Vaswani et al., 2017)",
        "BERT: Pre-training of Deep Bidirectional Transformers (Devlin et al., 2018)",
        "Language Models are Few-Shot Learners (Brown et al., 2020)"
    ],
    "Vision Transformers": [
        "An Image is Worth 16x16 Words (Dosovitskiy et al., 2020)",
        "Masked Autoencoders Are Scalable Vision Learners (He et al., 2021)",
        "Scaling Vision Transformers (Zhai et al., 2021)"
    ],
    "Geospatial Foundation Models": [
        "Prithvi: A Foundation Model for Earth Observation (IBM/NASA, 2023)",
        "SatMAE: Masked Autoencoders for Satellite Imagery (Cong et al., 2022)",
        "Clay: A Foundation Model for Earth Observation (Made with Clay, 2024)"
    ]
}

print("Essential References:")
print("="*40)

for category, papers in references.items():
    print(f"\n{category}:")
    for paper in papers:
        print(f"  • {paper}")

## Summary

Key concepts for foundation model architectures:
- **Historical Evolution**: From symbolic AI to transformer-based foundation models
- **Architecture Comparison**: LLMs use discrete tokenization, GFMs use continuous patch embeddings
- **Development Pipeline**: 9-step process with domain-specific adaptations
- **Scaling Trends**: LLMs scale in parameters/context, GFMs scale in spectral/spatial/temporal dimensions
- **Training Objectives**: Next-token prediction vs. masked patch reconstruction
- **Deployment Considerations**: Text streaming vs. spatial tiling and batch inference
- **Course Integration**: Weekly progression from data processing to deployment